<img src="https://habrastorage.org/webt/ia/m9/zk/iam9zkyzqebnf_okxipihkgjwnw.jpeg" />
    
**<center>[mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course** </center><br>
Author: [Yury Kashnitskiy](https://yorko.github.io) (@yorko). [mlcourse.ai](https://mlcourse.ai) is powered by [OpenDataScience (ods.ai)](https://ods.ai/) © 2017—2021

## <center>Assignment #6. Task</center><a class="tocSkip">
### <center> Beating benchmarks in "How good is your Medium article?"</center><a class="tocSkip">
    
[Competition](https://www.kaggle.com/c/how-good-is-your-medium-article). The task is to beat "Assignment 6 baseline" (~1.45 Public LB score). You can refer to [this simple Ridge baseline](https://www.kaggle.com/kashnitsky/ridge-countvectorizer-baseline?rvi=1).

*For discussions, please stick to [ODS Slack](https://opendatascience.slack.com/), channel __#mlcourse_ai_eng__, pinned thread __#a6_bonus__. If you are sure that something is not 100% correct, please leave your feedback there*

-----

# Imports

In [1]:
# !pip install xgboost nltk dill optuna

In [1]:
import json
import os
import pickle
from collections import defaultdict
from functools import partial
from html.parser import HTMLParser
from pathlib import Path

import dill as pickle
import numpy as np
import optuna
import pandas as pd
import xgboost as xgb
from nltk import TweetTokenizer
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from tqdm.notebook import tqdm

# Params

In [2]:
SEED = 42
PATH_TO_DATA = "data"
PATH_TO_SAVE_DIR = "prepared_data"

# Cleaning HTML

The following code will help to throw away all HTML tags from an article content.

In [3]:
from html.parser import HTMLParser


class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.fed = []

    def handle_data(self, d):
        self.fed.append(d)

    def get_data(self):
        return "".join(self.fed)


def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

# Reading JSON

Supplementary function to read a JSON line without crashing on escape characters.

In [4]:
def read_json_line(line=None):
    result = None
    try:
        result = json.loads(line)
    except Exception as e:
        # Find the offending character index:
        idx_to_replace = int(str(e).split(" ")[-1].replace(")", ""))
        # Remove the offending character:
        new_line = list(line)
        new_line[idx_to_replace] = " "
        new_line = "".join(new_line)
        return read_json_line(line=new_line)
    return result

Extract features `content`, `published`, `title` and `author`, write them to separate files for train and test sets.

In [5]:
def check_file(filename):
    file = Path(filename)
    return file.is_file()

In [6]:
def read_from_disk(features, path_to_save, prefix):
    reload_files = False
    d = defaultdict()
    for feature in features:
        filename = os.path.join(path_to_save, prefix + "_" + feature + ".txt")
        if check_file(filename):
            print(f"Reading from disk {filename}")
            with open(filename, "rb") as fp:
                d[feature] = pickle.load(fp)
        else:
            reload_files = True

    return reload_files, d

In [7]:
def extract_features_and_write(path_to_data, path_to_save, inp_filename, is_train=True):
    titles = []
    contents = []
    dates = []

    authors = []
    features = ["content", "published", "title", "author"]
    prefix = "train" if is_train else "test"

    Path(path_to_save).mkdir(parents=True, exist_ok=True)

    reload_files, d = read_from_disk(features, path_to_save, prefix)

    if reload_files:

        feature_files = [
            open(os.path.join(path_to_save, "{}_{}.txt".format(prefix, feat)), "w", encoding="utf-8")
            for feat in features
        ]

        with open(os.path.join(path_to_data, inp_filename), encoding="utf-8") as inp_json_file:

            for line in tqdm(inp_json_file, desc=f"Reading {prefix} json files"):
                json_data = read_json_line(line)

                title = json_data["title"].replace("\n", " ").replace("\t", " ").replace("\r", " ").replace("\xa0", " ")
                content = strip_tags(
                    json_data["content"].replace("\n", " ").replace("\t", " ").replace("\r", " ").replace("\xa0", " ")
                )
                published = json_data["published"]
                author = json_data["meta_tags"]["author"]
                authors_name = json_data["meta_tags"]["author"]

                titles.append(title)
                contents.append(content)
                dates.append(published)
                authors.append(authors_name)

        d = {"content": contents, "published": dates, "title": titles, "author": authors}
        for feature in features:
            filename = prefix + "_" + feature + ".txt"
            with open(os.path.join(path_to_save, filename), "wb") as fp:
                pickle.dump(d[feature], fp)
    else:
        titles, contents, dates, authors = d["title"], d["content"], d["published"], d["author"]

    return titles, contents, dates, authors

Download the [competition data](https://www.kaggle.com/c/how-good-is-your-medium-article/data) and place it where it's convenient for you. You can modify the path to data below.

In [8]:
train_titles, train_contents, train_dates, train_authors = extract_features_and_write(
    PATH_TO_DATA, PATH_TO_SAVE_DIR, "train.json", is_train=True
)
test_titles, test_contents, test_dates, test_authors = extract_features_and_write(
    PATH_TO_DATA, PATH_TO_SAVE_DIR, "test.json", is_train=False
)

Reading from disk prepared_data\train_content.txt
Reading from disk prepared_data\train_published.txt
Reading from disk prepared_data\train_title.txt
Reading from disk prepared_data\train_author.txt
Reading from disk prepared_data\test_content.txt
Reading from disk prepared_data\test_published.txt
Reading from disk prepared_data\test_title.txt
Reading from disk prepared_data\test_author.txt


# Feature engineering

**Add the following groups of features:**
* Tf-Idf with article content:
  * ngram_range=(1, 2)
  * max_features=100000
* Tf-Idf with article titles:
  * ngram_range=(1, 2)
  * max_features=100000
* Time features: 
  * publication hour, 
  * time of the day 
  * weekend or not
* Bag of authors  
i.e. One-Hot-Encoded author names

In [9]:
TITLE_NGRAMS = (1, 2)  # for tf-idf on titles
CONTENT_NGRAMS = (1, 2)  # for tf-idf on contents
MAX_FEATURES = 100_000  # for tf-idf

MEAN_TEST_TARGET = 4.33328  # what we got by submitting all zeros

RIDGE_WEIGHT = 0.6  # weight of Ridge predictions in a blend with XGBoost

## Tokenizer

In [10]:
tokenizer = TweetTokenizer()
assert tokenizer.tokenize("Now I'm a man") == ["Now", "I'm", "a", "man"]

In [11]:
def tokenize(text):
    return tokenizer.tokenize(text)

## Doing TF-IDF vectorization for articles

In [12]:
art_vec_params = {
    "ngram_range": CONTENT_NGRAMS,
    "max_features": MAX_FEATURES,
    "tokenizer": tokenize,
    "stop_words": ENGLISH_STOP_WORDS,
}

In [13]:
article_vectorizer_params_str = f"{art_vec_params['ngram_range']}_{art_vec_params['max_features']}"
article_vec_name = f"vectorizer_article_{article_vectorizer_params_str}.pickle"

In [14]:
%%time
article_vec_is_saved = check_file(article_vec_name)
if article_vec_is_saved:
    vectorizer_article = pickle.load(open(article_vec_name, "rb"))
else:
    vectorizer_article = TfidfVectorizer(**article_vectorizer_params)
    vectorizer_article.fit(train_contents)
    pickle.dump(vectorizer_article, open(article_vec_name, "wb"))

X_train_article = vectorizer_article.transform(train_contents)
X_test_article = vectorizer_article.transform(test_contents)

CPU times: total: 12min 25s
Wall time: 12min 26s


## Doing TF-IDF vectorization for titles

In [15]:
title_vec_params = {
    "ngram_range": TITLE_NGRAMS,
    "max_features": MAX_FEATURES,
    "tokenizer": tokenize,
    "stop_words": ENGLISH_STOP_WORDS,
}

In [16]:
title_vectorizer_params_str = f"{title_vec_params['ngram_range']}_{title_vec_params['max_features']}"
title_vec_name = f"vectorizer_title_{title_vectorizer_params_str}.pickle"

In [17]:
%%time
title_vec_is_saved = check_file(title_vec_name)
if title_vec_is_saved:
    vectorizer_title = pickle.load(open(title_vec_name, "rb"))
else:
    vectorizer_title = TfidfVectorizer(**title_vec_params)
    vectorizer_title.fit(train_titles)
    pickle.dump(vectorizer_title, open(title_vec_name, "wb"))

X_train_title = vectorizer_title.transform(train_titles)
X_test_title = vectorizer_title.transform(test_titles)

CPU times: total: 6.27 s
Wall time: 6.27 s


## Preparing time features

In [18]:
# Time features
def add_time_features(dates):
    scaler = StandardScaler()
    hour = scaler.fit_transform(np.array([date.hour for date in dates]).reshape(-1, 1))
    weekday = scaler.fit_transform(np.array([date.weekday() for date in dates]).reshape(-1, 1))
    morning = scaler.fit_transform(((hour >= 7) & (hour <= 11)).astype("int").reshape(-1, 1))
    day = scaler.fit_transform(((hour >= 12) & (hour <= 18)).astype("int").reshape(-1, 1))
    evening = scaler.fit_transform(((hour >= 19) & (hour <= 23)).astype("int").reshape(-1, 1))
    night = scaler.fit_transform(((hour >= 0) & (hour <= 6)).astype("int").reshape(-1, 1))
    weekend_temp = np.array([date.weekday() for date in dates]).reshape(-1, 1)
    weekend = scaler.fit_transform(((weekend_temp >= 5) & (weekend_temp <= 6)).astype("int").reshape(-1, 1))

    feature_names = ["morning", "day", "evening", "night", "weekday"]
    time_features = pd.DataFrame(
        list(zip(morning.flatten(), day.flatten(), evening.flatten(), night.flatten(), weekend.flatten())),
        columns=feature_names,
    )
    sparse_time_features = csr_matrix(time_features.values)
    return sparse_time_features, feature_names

In [19]:
train_times = pd.to_datetime([date["$date"] for date in train_dates])
test_times = pd.to_datetime([date["$date"] for date in test_dates])

X_train_time_features_sparse, time_feature_names = add_time_features(train_times)
X_test_time_features_sparse, _ = add_time_features(test_times)

## Doing bag of authors

In [20]:
authors = np.unique(train_authors + test_authors)
enc = OneHotEncoder(handle_unknown="ignore")
enc.fit(authors.reshape(-1, 1))
enc.categories_
X_train_author_sparse = enc.transform(np.array(train_authors).reshape(-1, 1)).toarray()
X_test_author_sparse = enc.transform(np.array(test_authors).reshape(-1, 1)).toarray()

## Preparing additional features

In [21]:
train_len = [len(article) for article in train_contents]
test_len = [len(article) for article in test_contents]
scaler = StandardScaler()

X_train_len_sparse = scaler.fit_transform(np.array(train_len).reshape(-1, 1))
X_test_len_sparse = scaler.fit_transform(np.array(test_len).reshape(-1, 1))

## Joining features

**Join all sparse matrices.**

In [22]:
X_train_sparse = hstack(
    [X_train_article, X_train_title, X_train_author_sparse, X_train_time_features_sparse, X_train_len_sparse]
).tocsr()

X_test_sparse = hstack(
    [X_test_article, X_test_title, X_test_author_sparse, X_test_time_features_sparse, X_test_len_sparse]
).tocsr()

### Saving dataset for training

In [29]:
pickle.dump(X_trapickle, open("X_train_sparse.pickle", "wb"))
pickle.dump(X_test_sparse, open("X_test_sparse.pickle", "wb"))
pickle.dump(y_train, open("y_train.pickle", "wb"))

### Reading dataset from disk

In [8]:
X_train_sparse = pickle.load(open("X_train_sparse.pickle", "rb"))
X_test_sparse = pickle.load(open("X_test_sparse.pickle", "rb"))
y_train = pickle.load(open("y_train.pickle", "rb"))

# Read train target and split data for validation

In [23]:
train_target = pd.read_csv(os.path.join(PATH_TO_DATA, "train_log1p_recommends.csv"), index_col="id")
y_train = train_target["log_recommends"].values

In [24]:
train_part_size = int(0.7 * train_target.shape[0])

X_train_part_sparse = X_train_sparse[:train_part_size, :]
y_train_part = y_train[:train_part_size]

X_valid_sparse = X_train_sparse[train_part_size:, :]
y_valid = y_train[train_part_size:]

# Training

## Ridge

### Hyperparameters tuning

In [66]:
# alpha_values = np.logspace(-2, 2, 40)
alpha_values = np.logspace(0, 0.1, 40)

logit_grid_searcher = GridSearchCV(
    estimator=Ridge(random_state=SEED),
    param_grid={"alpha": alpha_values},
    scoring="neg_mean_absolute_error",
    n_jobs=6,
    cv=3,
    verbose=3,
)

logit_grid_searcher.fit(X_train_sparse, y_train)

print(logit_grid_searcher.best_params_)

Fitting 3 folds for each of 40 candidates, totalling 120 fits
{'alpha': 1.1867492730446552}


In [77]:
logit_grid_searcher.best_score_

-1.1032014144706075

{'alpha': 1.1867492730446552}

### Training with best hyperparameters

In [67]:
ridge = Ridge(random_state=SEED, alpha=1.1867492730446552)
ridge.fit(X_train_sparse, y_train)
ridge_test_pred = ridge.predict(X_test_sparse)

## XGBoost

https://www.kaggle.com/code/hamzaghanmi/xgboost-catboost-using-optuna?scriptVersionId=94510532  
https://blog.cambridgespark.com/hyperparameter-tuning-in-xgboost-4ff9100a3b2f  
https://www.youtube.com/watch?v=5nYqK-HaoKY  
https://www.kaggle.com/code/prashant111/a-guide-on-xgboost-hyperparameters-tuning/notebook#2.-XGBoost-hyperparameters-  

### Tuning # 1

In [25]:
def optimize(trial, X, y):
    train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.20, random_state=SEED)

    dtrain = xgb.DMatrix(train_x, label=train_y)
    dvalid = xgb.DMatrix(valid_x, label=valid_y)

    param = {
        "seed": SEED,
        "verbosity": 1,
        "objective": "reg:pseudohubererror",
        "eval_metric": "mae",
        # gpu_hist for faster training
        "tree_method": "gpu_hist",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
        # defines how selective algorithm is.
        "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
        # learning rate should be tuned also
        "eta": trial.suggest_float("eta", 1e-8, 0.3, log=True),
        # the maximum depth of a tree
        "max_depth": trial.suggest_int("max_depth", 3, 6),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        # sampling for each split.
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.2, 1.0),
        # minimum child weight, larger the term more conservative the tree.
        "min_child_weight": trial.suggest_int("min_child_weight", 2, 10),
        # defines how selective algorithm is.
        "gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True),
    }

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    bst = xgb.train(param, dtrain, num_boost_round=10, evals=[(dtrain, 'train') , (dvalid, 'valid')])
    preds = bst.predict(dvalid)
    mae = mean_absolute_error(valid_y, preds)
    return mae

In [26]:
study = optuna.create_study(direction="minimize")
optimization_function = partial(optimize, X=X_train_sparse, y=y_train)
study.optimize(optimization_function, n_trials=100, timeout=600)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")

trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")

for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-05-14 19:38:56,659] A new study created in memory with name: no-name-886ec20b-2bf4-40bd-ae03-b555c85eb88d


[0]	train-mae:2.55233	valid-mae:2.54836
[1]	train-mae:2.55233	valid-mae:2.54836
[2]	train-mae:2.55233	valid-mae:2.54836
[3]	train-mae:2.55232	valid-mae:2.54835
[4]	train-mae:2.55232	valid-mae:2.54835
[5]	train-mae:2.55232	valid-mae:2.54835
[6]	train-mae:2.55232	valid-mae:2.54835
[7]	train-mae:2.55231	valid-mae:2.54835
[8]	train-mae:2.55231	valid-mae:2.54834
[9]	train-mae:2.55231	valid-mae:2.54834


[I 2022-05-14 19:39:04,808] Trial 0 finished with value: 2.5483408523720095 and parameters: {'booster': 'dart', 'grow_policy': 'lossguide', 'eta': 3.7770108294162213e-07, 'max_depth': 6, 'lambda': 0.0005710434639914917, 'alpha': 1.153364178878217e-06, 'subsample': 0.8749220448797093, 'colsample_bytree': 0.27517746843506635, 'min_child_weight': 5, 'gamma': 7.66891214645706e-08, 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 5.015019784891533e-07, 'skip_drop': 2.2897671762452735e-07}. Best is trial 0 with value: 2.5483408523720095.


[0]	train-mae:2.55233	valid-mae:2.54836
[1]	train-mae:2.55233	valid-mae:2.54836
[2]	train-mae:2.55233	valid-mae:2.54836
[3]	train-mae:2.55233	valid-mae:2.54836
[4]	train-mae:2.55233	valid-mae:2.54836
[5]	train-mae:2.55233	valid-mae:2.54836
[6]	train-mae:2.55233	valid-mae:2.54836
[7]	train-mae:2.55233	valid-mae:2.54836
[8]	train-mae:2.55233	valid-mae:2.54836
[9]	train-mae:2.55233	valid-mae:2.54836


[I 2022-05-14 19:39:20,790] Trial 1 finished with value: 2.5483608178954347 and parameters: {'booster': 'dart', 'grow_policy': 'depthwise', 'eta': 5.17482669985481e-08, 'max_depth': 6, 'lambda': 2.218577651417594e-06, 'alpha': 0.11590049305730092, 'subsample': 0.6432499359449777, 'colsample_bytree': 0.5378731314746252, 'min_child_weight': 8, 'gamma': 2.196056696121911e-05, 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 0.00427605742956224, 'skip_drop': 0.07688597486456346}. Best is trial 0 with value: 2.5483408523720095.


...

[0]	train-mae:2.53498	valid-mae:2.53099
[1]	train-mae:2.51721	valid-mae:2.51322
[2]	train-mae:2.49881	valid-mae:2.49466
[3]	train-mae:2.48047	valid-mae:2.47616
[4]	train-mae:2.46333	valid-mae:2.45892
[5]	train-mae:2.44728	valid-mae:2.44272
[6]	train-mae:2.43229	valid-mae:2.42769
[7]	train-mae:2.41784	valid-mae:2.41318
[8]	train-mae:2.40396	valid-mae:2.39920
[9]	train-mae:2.39069	valid-mae:2.38585


[I 2022-05-14 19:48:39,862] Trial 37 finished with value: 2.3858484776793545 and parameters: {'booster': 'gbtree', 'grow_policy': 'depthwise', 'eta': 0.0036982863724254293, 'max_depth': 4, 'lambda': 4.177606091377989e-06, 'alpha': 2.3549282239963276e-07, 'subsample': 0.6939722268719406, 'colsample_bytree': 0.5431450072158877, 'min_child_weight': 8, 'gamma': 0.32069999817484446}. Best is trial 31 with value: 1.3917488294785456.


[0]	train-mae:2.39551	valid-mae:2.38929
[1]	train-mae:2.29520	valid-mae:2.28954
[2]	train-mae:2.21119	valid-mae:2.20675
[3]	train-mae:2.14336	valid-mae:2.14063
[4]	train-mae:2.08682	valid-mae:2.08444
[5]	train-mae:2.03288	valid-mae:2.03165
[6]	train-mae:1.98414	valid-mae:1.98372
[7]	train-mae:1.93844	valid-mae:1.93920
[8]	train-mae:1.89501	valid-mae:1.89615
[9]	train-mae:1.85669	valid-mae:1.85811


[I 2022-05-14 19:49:10,162] Trial 38 finished with value: 1.85811438459204 and parameters: {'booster': 'dart', 'grow_policy': 'depthwise', 'eta': 0.0280117054049344, 'max_depth': 5, 'lambda': 0.04046810054265252, 'alpha': 4.1319871454466696e-05, 'subsample': 0.7734850314676558, 'colsample_bytree': 0.8038129584607341, 'min_child_weight': 6, 'gamma': 3.6662998865747404e-06, 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 1.3206848783228852e-05, 'skip_drop': 5.130919231681643e-06}. Best is trial 31 with value: 1.3917488294785456.


Number of finished trials:  39
Best trial:
  Value: 1.3917488294785456
  Params: 
    booster: dart
    grow_policy: depthwise
    eta: 0.19502384314948665
    max_depth: 4
    lambda: 0.3839413851896351
    alpha: 3.271548114884581e-05
    subsample: 0.8087637470753185
    colsample_bytree: 0.9828590616563115
    min_child_weight: 8
    gamma: 0.010458740824139907
    sample_type: uniform
    normalize_type: forest
    rate_drop: 4.6579551666720177e-08
    skip_drop: 0.00047103014517746846


Value: 1.3917488294785456  
  Params:   
    booster: dart  
    grow_policy: depthwise  
    eta: 0.19502384314948665  
    max_depth: 4  
    lambda: 0.3839413851896351  
    alpha: 3.271548114884581e-05  
    subsample: 0.8087637470753185  
    colsample_bytree: 0.9828590616563115  
    min_child_weight: 8  
    gamma: 0.010458740824139907  
    sample_type: uniform  
    normalize_type: forest  
    rate_drop: 4.6579551666720177e-08  
    skip_drop: 0.00047103014517746846  

### Tuning # 2

In [9]:
def optimize(trial, X, y):
    train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.20, random_state=SEED)

    dtrain = xgb.DMatrix(train_x, label=train_y)
    dvalid = xgb.DMatrix(valid_x, label=valid_y)

    param = {
        "seed": SEED,
        "verbosity": 0,
        "objective": "reg:pseudohubererror",
        "eval_metric": "mae",
        # gpu_hist for faster training
        "tree_method": "gpu_hist",
        # defines booster, gblinear for linear functions.
        "booster": "dart",
        # defines how selective algorithm is.
        "grow_policy": "depthwise",
        # learning rate should be tuned also
        "eta": trial.suggest_float("eta", 0.01, 0.3, log=True),
        # the maximum depth of a tree
        "max_depth": 4,
        # L2 regularization weight.
        "lambda": 0.3839413851896351,
        # L1 regularization weight.
        "alpha": 3.271548114884581e-05,
        # sampling ratio for training data.
        "subsample": 0.8087637470753185,
        # sampling according to each tree.
        "colsample_bytree": 0.9828590616563115,
        # sampling for each split.
        "colsample_bylevel": 0.9828590616563115,
        # minimum child weight, larger the term more conservative the tree.
        "min_child_weight": 8,
        # defines how selective algorithm is.
        "gamma": 0.010458740824139907,
    }

    param["sample_type"] = "uniform"
    param["normalize_type"] = "forest"
    param["rate_drop"] = 4.6579551666720177e-08
    param["skip_drop"] = 0.00047103014517746846

    bst = xgb.train(param, dtrain, num_boost_round=100, evals=[(dtrain, 'train') , (dvalid, 'valid')])
    preds = bst.predict(dvalid)
    mae = mean_absolute_error(valid_y, preds)
    return mae

In [11]:
study = optuna.create_study(direction="minimize")
optimization_function = partial(optimize, X=X_train_sparse, y=y_train)
study.optimize(optimization_function, n_trials=100, timeout=6000)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")

trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")

for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-05-14 20:15:53,117] A new study created in memory with name: no-name-994d91fb-9fe2-4d66-97bd-94a4af889299


[0]	train-mae:2.25237	valid-mae:2.24447
[1]	train-mae:2.11534	valid-mae:2.10749
[2]	train-mae:2.00908	valid-mae:2.00286
[3]	train-mae:1.91505	valid-mae:1.90999
[4]	train-mae:1.84119	valid-mae:1.83661
[5]	train-mae:1.77773	valid-mae:1.77424
[6]	train-mae:1.72232	valid-mae:1.71854
[7]	train-mae:1.67524	valid-mae:1.67262
[8]	train-mae:1.63467	valid-mae:1.63367
[9]	train-mae:1.59869	valid-mae:1.59921
[10]	train-mae:1.56618	valid-mae:1.56766
[11]	train-mae:1.53846	valid-mae:1.54136
[12]	train-mae:1.51342	valid-mae:1.51723
[13]	train-mae:1.49189	valid-mae:1.49569
[14]	train-mae:1.47275	valid-mae:1.47695
[15]	train-mae:1.45591	valid-mae:1.46106
[16]	train-mae:1.44002	valid-mae:1.44582
[17]	train-mae:1.42630	valid-mae:1.43286
[18]	train-mae:1.41397	valid-mae:1.42059
[19]	train-mae:1.40290	valid-mae:1.40993
[20]	train-mae:1.39297	valid-mae:1.40060
[21]	train-mae:1.38396	valid-mae:1.39207
[22]	train-mae:1.37523	valid-mae:1.38319
[23]	train-mae:1.36830	valid-mae:1.37642
[24]	train-mae:1.36158	val

[I 2022-05-14 20:19:56,110] Trial 0 finished with value: 1.2688200989606697 and parameters: {'eta': 0.05853850936494535}. Best is trial 0 with value: 1.2688200989606697.


[0]	train-mae:2.13170	valid-mae:2.11972
[1]	train-mae:1.94450	valid-mae:1.93430
[2]	train-mae:1.81546	valid-mae:1.80820
[3]	train-mae:1.71823	valid-mae:1.71460
[4]	train-mae:1.64527	valid-mae:1.64339
[5]	train-mae:1.58804	valid-mae:1.58746
[6]	train-mae:1.54279	valid-mae:1.54326
[7]	train-mae:1.50722	valid-mae:1.50903
[8]	train-mae:1.47782	valid-mae:1.48207
[9]	train-mae:1.45407	valid-mae:1.45831
[10]	train-mae:1.43341	valid-mae:1.43835
[11]	train-mae:1.41597	valid-mae:1.42186
[12]	train-mae:1.40207	valid-mae:1.40786
[13]	train-mae:1.38876	valid-mae:1.39514
[14]	train-mae:1.37769	valid-mae:1.38518
[15]	train-mae:1.36973	valid-mae:1.37811
[16]	train-mae:1.36241	valid-mae:1.37107
[17]	train-mae:1.35656	valid-mae:1.36543
[18]	train-mae:1.35137	valid-mae:1.36126
[19]	train-mae:1.34538	valid-mae:1.35609
[20]	train-mae:1.34055	valid-mae:1.35231
[21]	train-mae:1.33603	valid-mae:1.34814
[22]	train-mae:1.33230	valid-mae:1.34489
[23]	train-mae:1.32859	valid-mae:1.34233
[24]	train-mae:1.32467	val

[I 2022-05-14 20:22:13,620] Trial 1 finished with value: 1.2436840722598155 and parameters: {'eta': 0.10353223325237743}. Best is trial 1 with value: 1.2436840722598155.


...

[I 2022-05-14 21:15:58,767] Trial 25 finished with value: 1.2499675682923586 and parameters: {'eta': 0.08855139577249355}. Best is trial 10 with value: 1.234455828166958.


[0]	train-mae:2.32957	valid-mae:2.32255
[1]	train-mae:2.21146	valid-mae:2.20440
[2]	train-mae:2.11777	valid-mae:2.11160
[3]	train-mae:2.03863	valid-mae:2.03356
[4]	train-mae:1.96686	valid-mae:1.96314
[5]	train-mae:1.90431	valid-mae:1.90146
[6]	train-mae:1.85026	valid-mae:1.84870
[7]	train-mae:1.80268	valid-mae:1.80170
[8]	train-mae:1.75973	valid-mae:1.76037
[9]	train-mae:1.72086	valid-mae:1.72221
[10]	train-mae:1.68609	valid-mae:1.68829
[11]	train-mae:1.65497	valid-mae:1.65844
[12]	train-mae:1.62603	valid-mae:1.63090
[13]	train-mae:1.60051	valid-mae:1.60549
[14]	train-mae:1.57685	valid-mae:1.58256
[15]	train-mae:1.55587	valid-mae:1.56238
[16]	train-mae:1.53599	valid-mae:1.54304
[17]	train-mae:1.51797	valid-mae:1.52548
[18]	train-mae:1.50181	valid-mae:1.50909
[19]	train-mae:1.48702	valid-mae:1.49404
[20]	train-mae:1.47335	valid-mae:1.48026
[21]	train-mae:1.46075	valid-mae:1.46781
[22]	train-mae:1.44920	valid-mae:1.45638
[23]	train-mae:1.43802	valid-mae:1.44536
[24]	train-mae:1.42772	val

[I 2022-05-14 21:18:18,057] Trial 26 finished with value: 1.2872195867984781 and parameters: {'eta': 0.04241065290962887}. Best is trial 10 with value: 1.234455828166958.


[0]	train-mae:2.09964	valid-mae:2.09125
[1]	train-mae:1.90390	valid-mae:1.90533
[2]	train-mae:1.73416	valid-mae:1.72737
[3]	train-mae:1.63289	valid-mae:1.62971
[4]	train-mae:1.56062	valid-mae:1.56143
[5]	train-mae:1.51209	valid-mae:1.51506
[6]	train-mae:1.47620	valid-mae:1.47942
[7]	train-mae:1.44729	valid-mae:1.45106
[8]	train-mae:1.42366	valid-mae:1.42802
[9]	train-mae:1.40627	valid-mae:1.41151
[10]	train-mae:1.39235	valid-mae:1.39812
[11]	train-mae:1.38005	valid-mae:1.38640
[12]	train-mae:1.37082	valid-mae:1.37785
[13]	train-mae:1.36234	valid-mae:1.37056
[14]	train-mae:1.35440	valid-mae:1.36363
[15]	train-mae:1.34760	valid-mae:1.35835
[16]	train-mae:1.34065	valid-mae:1.35202
[17]	train-mae:1.33546	valid-mae:1.34808
[18]	train-mae:1.33133	valid-mae:1.34435
[19]	train-mae:1.32758	valid-mae:1.34079
[20]	train-mae:1.32424	valid-mae:1.33870
[21]	train-mae:1.31992	valid-mae:1.33515
[22]	train-mae:1.31558	valid-mae:1.33308
[23]	train-mae:1.31188	valid-mae:1.33053
[24]	train-mae:1.30803	val

KeyboardInterrupt: 

Number of finished trials:  3  
Best trial:  
  Value: 1.2337084043245454  
  Params:   
    eta: 0.13509680253299897  
    
Trial 10 finished with value: 1.234455828166958 and parameters: {'eta': 0.15406470353245602}.

Use tuned parameters and train xgboost on cpu on the whole dataset

In [71]:
import multiprocessing

In [32]:
xgb_reg = xgb.XGBRegressor(
    n_jobs=multiprocessing.cpu_count() // 2,
    num_round=XGB_TRAIN_ROUNDS,
    max_leaves=XGB_NUM_LEAVES,
    eval_metric="mae",
    objective="reg:pseudohubererror",
)
bst_xgb = xgb_reg.fit(X_train_sparse, y_train)
xgb_test_pred = bst_xgb.predict(X_test_sparse.astype(np.float32))

[19:49:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\Users\ryblo\AppData\Local\Temp\ipykernel_17936\2221456.py:9: RuntimeWarning: overflow encountered in expm1
  xgb_test_pred = np.expm1(bst_xgb.predict(X_test_sparse.astype(np.float32)))


## Training with all data

**Train the same Ridge with all available data, make predictions for the test set and form a submission file.**

In [48]:
mix_pred = XGB_WEIGHT * xgb_test_pred + RIDGE_WEIGHT * ridge_test_pred
mix_test_pred_modif = ridge_test_pred + MEAN_TEST_TARGET - y_train.mean()

NameError: name 'XGB_WEIGHT' is not defined

In [50]:
def write_submission_file(
    prediction,
    filename,
    path_to_sample=os.path.join(PATH_TO_DATA, "sample_submission.csv"),
):
    submission = pd.read_csv(path_to_sample, index_col="id")

    submission["log_recommends"] = prediction
    submission.to_csv(filename)

In [70]:
write_submission_file(mix_test_pred_modif, os.path.join(PATH_TO_DATA, "assignment6_medium_submission.csv"))

**Now's the time for dirty Kaggle hacks. Form a submission file with all zeros. Make a submission. What do you get if you think about it? How is it going to help you with modifying your predictions?**

In [18]:
write_submission_file(
    np.zeros_like(ridge_test_pred),
    os.path.join(PATH_TO_DATA, "medium_all_zeros_submission.csv"),
)

**Modify predictions in an appropriate way (based on your all-zero submission) and make a new submission.**

In [19]:
ridge_test_pred_modif = ridge_test_pred
# You code here (read-only in a JupyterBook, pls run jupyter-notebook to edit)

In [20]:
write_submission_file(
    ridge_test_pred_modif,
    os.path.join(PATH_TO_DATA, "assignment6_medium_submission_with_hack.csv"),
)

That's it for the assignment. In case you'd like to try some more ideas for improvement:

- Engineer good features, this is the key to success. Some simple features will be based on publication time, authors, content length and so on
- You may not ignore HTML and extract some features from there
- You'd better experiment with your validation scheme. You should see a correlation between your local improvements and LB score
- Try TF-IDF, ngrams, Word2Vec and GloVe embeddings
- Try various NLP techniques like stemming and lemmatization
- Tune hyperparameters. In our example, we've left only 50k features and used C=1 as a regularization parameter, this can be changed
- SGD and Vowpal Wabbit will train much faster
- Play around with blending and/or stacking. An intro is given in [this Kernel](https://www.kaggle.com/kashnitsky/ridge-and-lightgbm-simple-blending) by @yorko 
- And neural nets of course. We don't cover them in this course byt still transformer-based architectures will likely perform well in such types of tasks